In [26]:
import random

In [19]:
AGENTIC_MODULES = {
    "IO": {"cost": 1, "description": "Simple Input/Output Agent"},
    "CoT": {"cost": 3, "description": "Chain-of-Thought Reasoning Agent"},
    "ReAct": {"cost": 5, "description": "Reasoning+Action Agent (e.g., tool use)"},
    "Debate": {"cost": 8, "description": "Multi-Agent Debate for Complex Problems"},
    "Reflexion": {"cost": 4, "description": "Self-Reflection Agent"},
    "WebSearch": {"cost": 2, "description": "Web Searching Agent"},
    "EarlyExit": {"cost": 0, "description": "Exit sampling process"}
}

In [20]:
class SimpleMaASController:
    def __init__(self, modules):
        self.modules = modules

        self.difficulty_mapping = {
            "easy": {"preferred": ["IO"], "max_cost": 2},
            "medium": {"preferred": ["IO", "CoT", "ReAct"], "max_cost": 6},
            "hard": {"preferred": ["CoT", "ReAct", "Debate"], "max_cost": 15},
        }

    def sample_architecture(self, query_difficulty, cost_aware=True):
        if query_difficulty not in self.difficulty_mapping:
            print(f"Warning: Unknown difficulty '{query_difficulty}'. Defaulting to 'medium'.")
            query_difficulty = "medium"

        config = self.difficulty_mapping[query_difficulty]
        preferred_modules = config["preferred"]
        max_cost = config["max_cost"]

        sampled_architecture = []
        current_cost = 0


        initial_choice = random.choice([m for m in preferred_modules if m in self.modules])
        if self.modules[initial_choice]["cost"] <= max_cost or not cost_aware:
             sampled_architecture.append(initial_choice)
             current_cost += self.modules[initial_choice]["cost"]


        potential_additions = [m for m in self.modules if m not in sampled_architecture]
        random.shuffle(potential_additions)

        for module_name in potential_additions:
             module_cost = self.modules[module_name]["cost"]

             if not cost_aware or (current_cost + module_cost <= max_cost) :
                  if random.random() > 0.4:
                    sampled_architecture.append(module_name)
                    current_cost += module_cost

             if cost_aware and current_cost >= max_cost:
                  break

        if not sampled_architecture:

            cheapest_preferred = min(preferred_modules, key=lambda m: self.modules[m]["cost"])
            if self.modules[cheapest_preferred]["cost"] <= max_cost or not cost_aware:
                 sampled_architecture.append(cheapest_preferred)
            else:


                 absolute_cheapest = min(self.modules, key=lambda m: self.modules[m]["cost"])
                 sampled_architecture.append(absolute_cheapest)


        print(f"\n--- Sampling for Difficulty: {query_difficulty} (Cost Aware: {cost_aware}) ---")
        print(f"Target Max Cost: {'N/A' if not cost_aware else max_cost}")
        print(f"Sampled Architecture: {sampled_architecture}")
        print(f"Estimated Cost: {sum(self.modules[m]['cost'] for m in sampled_architecture)}")

        return sampled_architecture

In [21]:
#EXAMPLE
controller = SimpleMaASController(AGENTIC_MODULES)

print("Simulating MaAS Controller Sampling...")

Simulating MaAS Controller Sampling...


In [22]:
#EASY QUERY
controller.sample_architecture("easy", cost_aware=True)


--- Sampling for Difficulty: easy (Cost Aware: True) ---
Target Max Cost: 2
Sampled Architecture: ['IO']
Estimated Cost: 1


['IO']

In [23]:
#MEDIUM QUERY
controller.sample_architecture("medium", cost_aware=True)


--- Sampling for Difficulty: medium (Cost Aware: True) ---
Target Max Cost: 6
Sampled Architecture: ['ReAct', 'IO']
Estimated Cost: 6


['ReAct', 'IO']

In [24]:
#HARD QUERY
controller.sample_architecture("hard", cost_aware=True)


--- Sampling for Difficulty: hard (Cost Aware: True) ---
Target Max Cost: 15
Sampled Architecture: ['ReAct', 'Debate', 'WebSearch']
Estimated Cost: 15


['ReAct', 'Debate', 'WebSearch']

In [25]:
#UNAWARE
controller.sample_architecture("medium", cost_aware=False)


--- Sampling for Difficulty: medium (Cost Aware: False) ---
Target Max Cost: N/A
Sampled Architecture: ['IO', 'Debate', 'ReAct']
Estimated Cost: 14


['IO', 'Debate', 'ReAct']